In [ ]:
# The workflow for analyzing the routes begins with coordinate points used as origin locations.
import pandas as pd

df = pd.DataFrame({
    "city_name": ["Umeå", "Dresden", "Barcelona"],
    "region": ["europe", "europe", "europe"],
    "country": ["Sweden", "Germany", "Spain"],
})

# To sample nodes from the cities we call use a method from sample_nodes.py
import node_sampling

df = node_sampling.get_random_nodes_for_all_cities(df,min_distance_km=3)


display(df)

import os # for file operations

if not os.path.exists("example"):
    os.makedirs("example")
df.to_csv("example/city_samples.csv",index=False)


In [ ]:
from origin_graph import origin_graph # for creating the origin graph object
import os # for file operations
import ast # for parsing string to tuple
import pandas as pd # for reading the csv file
import osmnx as ox # for plotting the graph
"""
The origin graph object requires the following parameters for instantiation:
# Required parameters:
- origin_point: The coordinate point of the city as a tuple (latitude, longitude)
- distance_from_point: The distance from the origin point in meters to the sides of a bounding box.
- city_name: The name of the city
- network_type: The type of network to use. The default is 'drive'.

# The following parameters are optional:
- remove_parallel: A boolean to remove parallel edges in the graph. The default is False.
- simplify: A boolean to simplify the graph. The default is False.
- edge_attr_diff: The attribute to differentiate edges in the graph when simplifying the graph. The default is None.

# Downloading a graph and weighing it can take some time. To speed up the process, you can load a graph from a file.
- save_graphml: The path to save the graphml file.
- load_graphml: The path to a graphml file to load.
"""

folder = "example/local_origin_graphs"

if not os.path.exists(folder):
    os.makedirs(folder)

df = pd.read_csv("example/city_samples.csv")

df['graph_path'] = None

for index, row in df.iterrows():
    print(f"Creating origin graph for {row['city_name']} node id {row['node_id']}")
    latlon_point = ast.literal_eval(row['node_latlon'])
    og = origin_graph(origin_point=latlon_point, distance_from_point=2000,
                city_name=row["city_name"], network_type='drive', remove_parallel=True, simplify=True)
    
    graph_path = os.path.join(folder, f"{row['city_name']}_{row['node_id']}.graphml")
    print(f"Saving graph to {graph_path}")
    og.save_graph(graph_path)

    # plot the origin graph to see if something is obviously wrong
    ox.plot_graph(og.graph, node_color='blue', node_size=10, edge_linewidth=1, edge_color='black', bgcolor='white')

    # Update the corresponding column in the dataframe
    df.at[index, f'graph_path'] = graph_path

df.to_csv("example/city_samples.csv",index=False)

In [ ]:
# The next step is to add weights to the edges of the graph.

from origin_graph import origin_graph # for creating the origin graph object
import os # for file operations
import ast # for parsing string to tuple
import pandas as pd # for reading the csv file

folder = "example/local_origin_graphs"
df = pd.read_csv("example/city_samples.csv")

for index, row in df.iterrows():
    print(f"Adding weights to graph for {row['city_name']} node id {row['node_id']}")
    og = origin_graph.from_graphml(graphml_path=row['graph_path'])

    # Adding the decision complexity weights may take some time.
    og.add_weights('decision_complexity')
    og.add_weights('deviation_from_prototypical')
    og.add_weights('node_degree')
    og.add_weights('instruction_equivalent')

    # We can print the edge weights to see which have been added.
    print(og.edge_weights)

    # Save the graph with weights
    og.save_graph(row['graph_path'])

In [ ]:
from origin_graph import origin_graph # for creating the origin graph object
import os # for file operations
import ast # for parsing string to tuple
import pandas as pd # for reading the csv file
import psutil

import gc

print(f"Memory usage: {psutil.Process().memory_info().rss / 1024 / 1024:.2f} MB")
folder = "example/local_origin_graphs"
df = pd.read_csv("example/city_samples.csv")

origins_od_pair_data = []
for index, row in df.iterrows():
    print(f"Creating od-pairs for {row['city_name']} node id {row['node_id']}")
    og = origin_graph.from_graphml(graphml_path=row['graph_path'])

    """
    Parameters:
    - min_distance: The minimum distance in meters between the origin and destination points in great circle distance.
    - max distance: The maximum distance in meters between the origin and destination points in great circle distance.
    - sample_size: The number of origin-destination pairs to sample.

    Return: a dataframe with each row corresponding to an origin-destination pair.
    """
    og.create_od_pairs(min_radius=950, max_radius=1050, sample_size=144)
    od_pair_data = og.get_od_pair_data()
    origins_od_pair_data.append(od_pair_data)

gc.collect()
print(f"Memory usage: {psutil.Process().memory_info().rss / 1024 / 1024:.2f} MB")
try:
    origin_od_pairs = pd.concat(origins_od_pair_data)
    print(f"Number of origin-destination pairs: {len(origin_od_pairs)} in the dataset.")
    origin_od_pairs.to_csv("example/origin_od_pairs.csv")
except ValueError as e:
    print(f"Error: {e}")
